In [1]:
import pandas as pd 
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
#import pylab as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import statsmodels.api as sm
from sklearn.cross_validation import KFold
%matplotlib inline

###This notebook will focus on the analytical part of the 311 demographics analysis: that is, multilinear regressions between resident and working population socio-demographic attributes and  the number of 311 calls by type per capita, at the NTA level will be performed and analyzed

The first step of this process, is to perform linear regression with the demographical features as variables and the types of 311 services requests as dependant variables.

Outcomes and variables of this notebook:

1)  regresions_filtered is an internal dictionary: the types of requests are the keys, and the group of variables which after the cross validation got an R square value greater than 0.2 are elements of each key.

2) outcomes/table_1.csv is a csv fail, containing the type of service request, the group of variables and the R square value of the OLS regression after the cross validation

3)results_phase1 is an internal dictionary. The structure is: results_phase1[type_of_request][group_of_variables] and contains the R square value of each regression as well as a table with the values of the coefficients of the regression associated with each variable under the cathegory (coefficients under the 'mean' column and the standard deviation, obtained from the width of the confidence intervals, unter the std_dv column)

4) outcomes/user_profiles.csv and outcomes/user_profiles_coefficients.csv contain a table each with the type of requests as index and the group of complain as column. The value on each cell represents the variable associated with the highest coefficient on the OLS regression (in the file user_profiles.csv) and the value of this coefficient (in the file user_profiles_coefficients.csv )

5) OLS_regressions is an internal dicionary. It follows the structure OLS_regressions[type_of_request][group_of_variables]['total_var' or 'prediction']. 'total_var' contains the standard deviation of the group of variables (sum of the stand. dev from the  results_phase1 dictionary), while 'prediction' containg a table comparing the observed values and the predicted values for the number of requests for each type of request, following the regression values obtained for each group of variables.

6) At the end of the notebook, avg_predicionts_dict is a dicionary that averages the different predictions for each type of request. avg_predicionts_dict[type_of_request] gives access to a table with the observed and averaged predicted number of requests for each NTA.

7) In order to provide a better way to understand and to communicate the findings of step 6, outputs/predicted_values.csv and
outputs/observed_values.csv are two files that contain the predicted number of requests for each type as predicted by our model, and the observed values. 


In [2]:
#Upload the working population attributes, and the resident population attributes at the CT level.
#We keep both datasets separated in order to generate separate regressions per type of features
demographics_NTA_NYC_residents=pd.read_csv('datasets/demographics_nta_NYC_residents_compiled.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
demographics_NTA_NYC_workers=pd.read_csv('datasets/demographics_nta_NYC_workers_compiled.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
demographics_NTA_NYC=pd.merge(demographics_NTA_NYC_residents,demographics_NTA_NYC_workers,on='Neighborhood',how='inner')
print len(demographics_NTA_NYC_residents),len(demographics_NTA_NYC_workers),len(demographics_NTA_NYC)

195 194 194


In [3]:
#Upload the 311 calls by type, normalized by total resident population
calls_bytype_normalized=pd.read_csv('datasets/Call by type without normalization - NTA level.csv').drop('Unnamed: 0', axis=1)

In [4]:
#demographics_NTA_NYC_workers.columns

In [5]:
#Delete the total population from the dataset since we are not going to use it
#del(calls_bytype_normalized['Total Population'])

In [6]:
#callsbytype_attributes will be a dataframe combining all the information (demographics + calls by type)
callsbytype_attributes=pd.merge(calls_bytype_normalized,demographics_NTA_NYC, on='Neighborhood',how='inner')
print len(callsbytype_attributes), len(calls_bytype_normalized),len(demographics_NTA_NYC)

190 190 194


In [7]:
#callsbytype_attributes will be a dataframe combining all the information (demographics + calls by type)
#we created to separate frames, one for residents and one for workers
callsbytype_attributes_residents=pd.merge(calls_bytype_normalized,demographics_NTA_NYC_residents, on='Neighborhood',how='inner')
callsbytype_attributes_workers=pd.merge(calls_bytype_normalized,demographics_NTA_NYC_workers, on='Neighborhood',how='inner')
print len(callsbytype_attributes_residents), len(callsbytype_attributes_workers)

190 190


In [8]:
types_of_calls=calls_bytype_normalized.columns[:-1][1:]  #types of calls
#calls_bytype_normalized.columns[:-3]

In [9]:
callsbytype_attributes.columns

Index([u'Neighborhood', u'AGENCY', u'APPLIANCE', u'Adopt-A-Basket',
       u'Air Quality', u'Animal Abuse', u'Animal Facility - No Permit',
       u'Animal in a Park', u'Asbestos', u'BEST/Site Safety', 
       ...
       u'household income form 10 to 40_n',
       u'household income form 40 to 75_n', u'household income 75 and above_n',
       u'house value less than 100_n', u'house value for 100 to 500_n',
       u'house value 500 or more_n', u'rent less than 1000_n',
       u'rent bewteen 1000 and 2000_n', u'Transportation Other means_n',
       u'rent 2000 or more_n'],
      dtype='object', length=247)

###We are defining manually the variables, and the groups of variables

In [10]:
regressors=[u'Population under 18', u'population between 18 and 34',
       u'population between 35 to 64', u'population 65 and over',
       u'Population white', u'population black', u'Population asian',
       u'population hispanic', u'population other race',
       u'family households', u'nonfamily households',
       u'population education high school or less', u'population education bachelors',
       u'population education masters', u'population education phd',
        u'owner  occupied units',
       u'renter occupied units', u'transportation car', u'number of cars',
       u'transportation public', u'tranportation motorcycle',
        u'household income less than 40',
       u'household income from 40 to 75', u'household income 75 and above',
       u'house value less than 100', u'house value for 100 to 500',
       u'house value 500 or more', u'rent less than 1000',
       u'rent bewteen 1000 and 2000', u'rent 2000 or more',
       u'Transportation Other means', u'population between 18 and 34_n',
       u'population between 35 to 64_n', u'population 65 and over_n',
       u'Population white_n', u'population black_n', u'Population asian_n',
       u'population hispanic_n', u'population other _n',
       u'family households_n', u'nonfamily households_n',
       u'population education high school and less_n',
       u'population education bachelors_n', u'population education masters_n',
       u'population education phd_n', u'household income less than 10_n',
       u'owner  occupied units_n', u'renter occupied units_n',
       u'transportation car_n', u'transportation public_n',
       u'tranportation motorcycle_n', 
       u'household income less than 10_n',
       u'household income form 10 to 40_n',
       u'household income form 40 to 75_n', u'household income 75 and above_n',
       u'house value less than 100_n', u'house value for 100 to 500_n',
       u'house value 500 or more_n', u'rent less than 1000_n',
       u'rent bewteen 1000 and 2000_n',u'rent 2000 or more_n', u'Transportation Other means_n']

In [11]:
#Defining the selected features for the innitial resident lasso regression
#we will use this innitial regression to determine a selected list of types of complaints 
#that meet a certain threadshold to then perform a selective regression by feature types
resident_features = [u'Population under 18', u'population between 18 and 34',
       u'population between 35 to 64', u'population 65 and over', u'Population white', u'population black', u'Population asian',
       u'population hispanic', u'population other race', u'family households', u'nonfamily households', 
                     u'population education high school or less', 
       u'population education bachelors', u'population education masters', u'population education phd', 'owner  occupied units',  
       u'renter occupied units', u'transportation car', u'transportation public', 'tranportation motorcycle', 
       u'Transportation Other means', u'number of cars', u'household income less than 40', u'household income from 40 to 75', 
       u'household income 75 and above', u'house value less than 100', u'house value for 100 to 500',
       u'house value 500 or more', u'rent less than 1000', u'rent bewteen 1000 and 2000', u'rent 2000 or more']                     

In [12]:
#This is the list of selected feature types for the second step
age_residents=[u'Population under 18', u'population between 18 and 34',
       u'population between 35 to 64', u'population 65 and over']

race_residents=[u'Population white', u'population black', u'Population asian',
       u'population hispanic', u'population other race']

typeof_household_residents=[u'family households', u'nonfamily households']
education_residents= [u'population education high school or less', u'population education bachelors', u'population education masters', 
                      u'population education phd']
ownorrent_residents = ['owner  occupied units',  u'renter occupied units']
transportationtype_residents = [u'transportation car', u'transportation public', 'tranportation motorcycle',
                              'Transportation Other means']
#omit 'number of cars' from the regression 
income_residents = [u'household income less than 40', u'household income from 40 to 75', u'household income 75 and above']
housing_values_residents = [u'house value less than 100', u'house value for 100 to 500',
       u'house value 500 or more']
rent_residents=[u'rent less than 1000',u'rent bewteen 1000 and 2000', u'rent 2000 or more']
variable_groups_residents=['age_residents','race_residents','typeof_household_residents',
                          'education_residents','ownorrent_residents','transportationtype_residents',
                          'income_residents','housing_values_residents','rent_residents']

In [13]:
#This is the list of selected feature types for the second step
age_workers=[u'population between 18 and 34_n',
       u'population between 35 to 64_n', u'population 65 and over_n']

race_workers=[u'Population white_n', u'population black_n', u'Population asian_n',
       u'population hispanic_n', u'population other _n']

typeof_household_workers=[u'family households_n', u'nonfamily households_n']
education_workers= [u'population education high school and less_n', u'population education bachelors_n', u'population education masters_n', 
                      u'population education phd_n']
ownorrent_workers = ['owner  occupied units_n',  u'renter occupied units_n']
transportationtype_workers = [u'transportation car_n', u'transportation public_n', 'tranportation motorcycle_n',
                              'Transportation Other means_n']
income_workers = [u'household income less than 10_n',u'household income form 10 to 40_n', u'household income form 40 to 75_n', u'household income 75 and above_n']
housing_values_workers = [u'house value less than 100_n', u'house value for 100 to 500_n',
       u'house value 500 or more_n']       
rent_workers= [ u'rent less than 1000_n',
       u'rent bewteen 1000 and 2000_n', u'rent 2000 or more_n'] 
variable_groups_workers=['age_workers','race_workers','typeof_household_workers','education_workers','ownorrent_workers',
                        'transportationtype_workers','income_workers','housing_values_workers','rent_workers']

#STEP 1


In this step, we will get the R square values for every regression
then, the estimated average of requests by type is going to be obtained

In [14]:
variable_groups=variable_groups_residents+variable_groups_workers

In [15]:
#FIRST, I WILL RUN OLS OVER EVERY TYPE OF COMPLAIN
ols_coefficients_dict={}
for typeof in types_of_calls:
    ols_coefficients_dict[typeof]={}
    #filter population
    for data_group in variable_groups:
        ols_coefficients_dict[typeof][data_group]={}
        features=eval(data_group)           
        A2=np.append(np.append(features ,typeof),'Neighborhood')   #selection of columns
        myframe1=callsbytype_attributes[A2].fillna(value=0)  
        X=myframe1[features]
        Y=myframe1[typeof]
        ####CROSS-VALIDATION
        kf = KFold(n=len(myframe1), n_folds=10, shuffle=True, random_state=21)
        r2store=[]
        for train_index, test_index in kf:
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            Y_train, Y_test = Y.loc[train_index], Y.loc[test_index]
            ordinary_LS=sm.OLS(Y_train, X_train).fit()
            pred_test=ordinary_LS.predict(X_test)
            if np.var(Y_test)==0:
                vv=0.000003
            else:
                vv=np.var(Y_test)            
            R2_OS = (1 - np.var(pred_test - Y_test) / vv)
            r2store.append(R2_OS)
        rsq=np.mean(r2store)
        #ols_coefficients_dict[typeof][data_group]['coefficients']=ordinary_LS.params
        #ols_coefficients_dict[typeof][data_group]['interval']=ordinary_LS.conf_int()
        ols_coefficients_dict[typeof][data_group]['Rsquared']=rsq

###Table 1

In [16]:
#In order to use it later, I will save two csv files: one with the name of the top features and another one with the values of 
#coefficients of those features.
table_r_sq = pd.DataFrame(index=ols_coefficients_dict.keys(), columns=variable_groups)
for typeof in ols_coefficients_dict.keys():
    for groupof in variable_groups:
        rsq=ols_coefficients_dict[typeof][groupof]['Rsquared']
        table_r_sq.loc[typeof,groupof]=rsq

In [17]:
#table_r_sq

In [18]:
#couples of types of requests and group of coefficients with r-sq>0.2
regresions_filtered={}
for typeof in types_of_calls:
    regresions_filtered[typeof]=[]
    for data_group in variable_groups:
        if table_r_sq.loc[typeof,data_group]>0.2:
            regresions_filtered[typeof].append(data_group)
for typeof in types_of_calls:
    if len(regresions_filtered[typeof])==0:
        regresions_filtered.pop(typeof, None)

In [19]:
regresions_filtered

{'APPLIANCE': ['age_residents',
  'race_residents',
  'ownorrent_residents',
  'transportationtype_residents',
  'income_residents',
  'rent_residents'],
 'Air Quality': ['age_residents',
  'race_residents',
  'typeof_household_residents',
  'education_residents',
  'ownorrent_residents',
  'transportationtype_residents',
  'income_residents',
  'rent_residents',
  'education_workers',
  'ownorrent_workers',
  'transportationtype_workers'],
 'Animal Abuse': ['age_residents',
  'race_residents',
  'typeof_household_residents',
  'education_residents',
  'transportationtype_residents',
  'income_residents',
  'rent_residents'],
 'Asbestos': ['age_residents',
  'typeof_household_residents',
  'education_residents',
  'ownorrent_residents',
  'transportationtype_residents',
  'income_residents',
  'rent_residents'],
 'Bike/Roller/Skate Chronic': ['age_residents',
  'typeof_household_residents',
  'education_residents',
  'ownorrent_residents',
  'transportationtype_residents',
  'income_re

In [20]:
table_1 = pd.DataFrame(index=range(0,len(table_r_sq)), columns=['Type of complain','Group of features','R square'])
i=0
for typeof in regresions_filtered.keys():
    for groupof in regresions_filtered[typeof]:
        table_1.loc[i,'Type of complain']=typeof
        table_1.loc[i,'Group of features']=groupof
        table_1.loc[i,'R square']=ols_coefficients_dict[typeof][groupof]['Rsquared']
        i=i+1

In [21]:
table_1.sort('Type of complain',ascending=True,inplace=True)
table_1.to_csv('outputs/table_1.csv')

In [22]:
table_1.head()

,Type of complain,Group of features,R square
118,APPLIANCE,ownorrent_residents,0.3275242
117,APPLIANCE,race_residents,0.612347
119,APPLIANCE,transportationtype_residents,0.2459905
120,APPLIANCE,income_residents,0.5258196
121,APPLIANCE,rent_residents,0.5806156


Now, we will train OLS over the entire set, for all the types of requests and groups of features from table 1 (that is, those with satisfactory R2)

In [23]:
coefficients_complete_table=pd.DataFrame(index=np.unique(table_1['Type of complain']), columns=regressors)
coefficients_std_complete_table=pd.DataFrame(index=np.unique(table_1['Type of complain']), columns=regressors)
results_phase1={}
for typeof in regresions_filtered.keys():
    results_phase1[typeof]={}
    for data_group in regresions_filtered[typeof]:
        results_phase1[typeof][data_group]={}
        features=eval(data_group)           
        A2=np.append(np.append(features ,typeof),'Neighborhood')   #selection of columns
        myframe1=callsbytype_attributes[A2].fillna(value=0)   
        X=myframe1[features]
        Y=myframe1[typeof]
        ordinary_LS=sm.OLS(Y, X).fit()
        rsq=ordinary_LS.rsquared
        table=ordinary_LS.conf_int()
        table['std_dv']=(table[1]-table[0])/4
        table['mean']=ordinary_LS.params
        table.drop([0,1],inplace=True,axis=1)
        results_phase1[typeof][data_group]['coefficients']=table
        results_phase1[typeof][data_group]['Rsquared']=rsq
        for feat in features:
            coefficients_complete_table.loc[typeof,feat]=results_phase1[typeof][data_group]['coefficients'].loc[feat,'mean']
            coefficients_std_complete_table.loc[typeof,feat]=results_phase1[typeof][data_group]['coefficients'].loc[feat,'std_dv']

In [24]:
coefficients_std_complete_table.head()

,Population under 18,population between 18 and 34,population between 35 to 64,population 65 and over,Population white,population black,Population asian,population hispanic,population other race,family households,...,household income form 10 to 40_n,household income form 40 to 75_n,household income 75 and above_n,house value less than 100_n,house value for 100 to 500_n,house value 500 or more_n,rent less than 1000_n,rent bewteen 1000 and 2000_n,rent 2000 or more_n,Transportation Other means_n
APPLIANCE,0.0009714331,0.001096273,0.001484194,0.0024011,0.0001569173,0.0001833691,0.0003822683,0.0001986578,0.001594724,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Air Quality,0.0006608248,0.0007457484,0.001009634,0.001633367,0.0001509091,0.0001763482,0.0003676319,0.0001910515,0.001533665,0.0003264036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003453515
Animal Abuse,0.0005329376,0.0006014263,0.000814243,0.001317267,0.0001034804,0.0001209243,0.0002520901,0.0001310066,0.001051654,0.0003227161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Asbestos,0.0002115581,0.0002387457,0.0003232267,0.00052291,NaN,NaN,NaN,NaN,NaN,0.0001021091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bike/Roller/Skate Chronic,6.507966e-05,7.344315e-05,9.943126e-05,0.000160858,NaN,NaN,NaN,NaN,NaN,3.087606e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
coefficients_complete_table.to_csv('outputs/step1_coefficients_complete_table.csv')
coefficients_std_complete_table.to_csv('outputs/coefficients_std_complete_table.csv')

In [26]:
results_phase1['APPLIANCE']['age_residents']['coefficients']

,std_dv,mean
Population under 18,0.000971,0.005266
population between 18 and 34,0.001096,0.003523
population between 35 to 64,0.001484,-0.000490
population 65 and over,0.002401,-0.005141


###For the visualization tool, the results on dictionary 'results_phase1' can be stored as the user profile in a csv file

In [27]:
user_profile_dict={}
for typeof in results_phase1.keys():
    user_profile_dict[typeof]={}
    for groupof in results_phase1[typeof].keys():
        table=results_phase1[typeof][groupof]['coefficients']
        max_feature=table[table['mean']==np.max(table['mean'])].index[0]
        max_coefficient=table[table['mean']==np.max(table['mean'])]['mean'][0]
        user_profile_dict[typeof][groupof]={max_feature:max_coefficient}

user_profiles = pd.DataFrame(index=results_phase1.keys(), columns=variable_groups)
user_profiles_coef = pd.DataFrame(index=results_phase1.keys(), columns=variable_groups)

for typeof in user_profile_dict.keys():
    for groupof in user_profile_dict[typeof].keys():
        user_feature=user_profile_dict[typeof][groupof].keys()[0]
        user_coeff=user_profile_dict[typeof][groupof].values()[0]
        user_profiles.loc[typeof,groupof]=user_feature
        user_profiles_coef.loc[typeof,groupof]=user_coeff

In [28]:
user_profiles

,age_residents,race_residents,typeof_household_residents,education_residents,ownorrent_residents,transportationtype_residents,income_residents,housing_values_residents,rent_residents,age_workers,race_workers,typeof_household_workers,education_workers,ownorrent_workers,transportationtype_workers,income_workers,housing_values_workers,rent_workers
Senior Center Complaint,NaN,NaN,NaN,NaN,renter occupied units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lead,population between 18 and 34,Population white,nonfamily households,population education masters,owner occupied units,tranportation motorcycle,household income 75 and above,NaN,rent 2000 or more,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WATER LEAK,Population under 18,population black,NaN,NaN,renter occupied units,NaN,household income less than 40,NaN,rent less than 1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bus Stop Shelter Placement,NaN,Population white,nonfamily households,population education masters,owner occupied units,transportation public,household income 75 and above,NaN,rent 2000 or more,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Found Property,population between 18 and 34,NaN,nonfamily households,population education bachelors,NaN,tranportation motorcycle,household income 75 and above,NaN,rent 2000 or more,population between 18 and 34_n,population other _n,nonfamily households_n,population education phd_n,renter occupied units_n,transportation public_n,household income less than 10_n,house value less than 100_n,rent less than 1000_n
For Hire Vehicle Complaint,population between 18 and 34,population other race,nonfamily households,NaN,renter occupied units,Transportation Other means,household income less than 40,NaN,rent 2000 or more,population between 35 to 64_n,NaN,NaN,NaN,renter occupied units_n,NaN,NaN,NaN,NaN
New Tree Request,NaN,NaN,NaN,NaN,NaN,tranportation motorcycle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unsanitary Pigeon Condition,population 65 and over,NaN,nonfamily households,population education phd,owner occupied units,tranportation motorcycle,household income from 40 to 75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Consumer Complaint,population between 18 and 34,population other race,nonfamily households,population education bachelors,renter occupied units,Transportation Other means,household income less than 40,NaN,rent 2000 or more,NaN,NaN,NaN,NaN,renter occupied units_n,NaN,NaN,NaN,NaN
Other Enforcement,population between 18 and 34,population other race,nonfamily households,population education bachelors,owner occupied units,tranportation motorcycle,household income 75 and above,house value 500 or more,rent 2000 or more,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
user_profiles.to_csv('outputs/user_profiles.csv')
user_profiles_coef.to_csv('outputs/user_profiles_coefficients.csv')

The coefficients obtained from the regression follow the formula:

$$S_q (A,t) = \sum_{k=1}^{s_q} P_q (A,k) x_{q,k,t} + \sum_{k=1}^{s_q} C_q (A,k) y_{q,k,t}$$

where $q$ represents each group of variables and $k$ is each variable, $x_{q,k,t}$ and $y_{q,k,t}$ are the coefficients obtained from the regressions.

The next step, consists on running the OLS model for the variables with sufficient R square values using the entire set as training and getting a prediction $\hat{S_q (A,t) }$ that is, a prediction for the total number of service requests of a certain type for each area based on the cathegory $q$


In [30]:
OLS_regressions={}
for typeof in regresions_filtered.keys():
    OLS_regressions[typeof]={}
    for data_group in regresions_filtered[typeof]:
        OLS_regressions[typeof][data_group]={}
        features=eval(data_group)           
        A2=np.append(np.append(features ,typeof),'Neighborhood')   #selection of columns
        myframe1=callsbytype_attributes[A2].fillna(value=0)  
        X=myframe1[features]
        Y=myframe1[typeof]
        ordinary_LS=sm.OLS(Y, X).fit()
        predicted_vals=ordinary_LS.predict(X)
        predictions_frame= pd.DataFrame(index=range(0,len(X)), columns=['NTA','observed_req','predicted_req'])
        predictions_frame['NTA']=myframe1['Neighborhood']
        predictions_frame['observed_req']=Y
        predictions_frame['predicted_req']=list(predicted_vals)
        OLS_regressions[typeof][data_group]['prediction']=predictions_frame
        total_var=np.sum(results_phase1[typeof][data_group]['coefficients'].std_dv)
        OLS_regressions[typeof][data_group]['total_var']=total_var

In [31]:
OLS_regressions['APPLIANCE']['age_residents']['total_var']

0.005953000450465134

In [32]:
OLS_regressions['APPLIANCE']['age_residents']['prediction'].head()

,NTA,observed_req,predicted_req
0,Allerton-Pelham Gardens,13,27.783434
1,Annadale-Huguenot-Prince's Bay-Eltingville,4,24.369890
2,Arden Heights,3,25.016549
3,Astoria,51,85.689254
4,Auburndale,6,14.213282


####NEXT STEP: average those estimates over all suitable q

that follows the formula: $$\hat{S} (A,t) = \frac{\sum_{q} S_q (A,t) \sigma _{q,t} ^{-2}}{\sum_{q} \sigma _{q,t} ^{-2}}$$

In [33]:
avg_predictions_frame= pd.DataFrame(index=range(0,len(X)), columns=['NTA','observed_req','predicted_req'])

In [34]:
avg_predicionts_dict={}
for typeof in regresions_filtered.keys():
    avg_predicionts_dict[typeof]={}
    vars_list=[]
    avg_predictions_frame= pd.DataFrame(index=range(0,len(X)), columns=['NTA','observed_req','predicted_req'])
    temporal_frame= pd.DataFrame(index=range(0,len(OLS_regressions[typeof][OLS_regressions[typeof].keys()[0]]['prediction'])))
    temporal_frame['NTA']=OLS_regressions[typeof][OLS_regressions[typeof].keys()[0]]['prediction'].NTA
    avg_predictions_frame['NTA']=temporal_frame['NTA']
    avg_predictions_frame['observed_req']=OLS_regressions[typeof][OLS_regressions[typeof].keys()[0]]['prediction'].observed_req
    for data_group in regresions_filtered[typeof]:
        my_var=OLS_regressions[typeof][data_group]['total_var']
        if my_var==0:
            std_e=0.00001
        else:
            std_e=(my_var**-2)
        vars_list.append(std_e)
        predicted_weighted=(std_e)*OLS_regressions[typeof][data_group]['prediction'].predicted_req
        temporal_frame[data_group]=predicted_weighted
    avg_predictions_frame['predicted_req']=(temporal_frame.sum(axis=1))/np.sum(vars_list)
    avg_predicionts_dict[typeof]=avg_predictions_frame
#predicted_av=predicted_weighted/np.sum(vars_list)
        

In [35]:
observed_table=pd.DataFrame(index=avg_predicionts_dict.keys(),columns=callsbytype_attributes['Neighborhood'])
predicted_table=pd.DataFrame(index=avg_predicionts_dict.keys(),columns=callsbytype_attributes['Neighborhood'])
for typeof in avg_predicionts_dict.keys():
    table=avg_predicionts_dict[typeof]
    for nta in callsbytype_attributes['Neighborhood']:
        observed_table.loc[typeof,nta]=table[table.NTA==nta].observed_req.sum()
        predicted_table.loc[typeof,nta]=table[table.NTA==nta].predicted_req.sum()
predicted_table.head()

Neighborhood,Allerton-Pelham Gardens,Annadale-Huguenot-Prince's Bay-Eltingville,Arden Heights,Astoria,Auburndale,Baisley Park,Bath Beach,Battery Park City-Lower Manhattan,Bay Ridge,Bayside-Bayside Hills,...,Williamsbridge-Olinville,Williamsburg,Windsor Terrace,Woodhaven,Woodlawn-Wakefield,Woodside,Yorkville,park-cemetery-etc-Bronx,park-cemetery-etc-Manhattan,park-cemetery-etc-Queens
Senior Center Complaint,0.6757804,0.5157766,0.4348189,3.519637,0.4432801,0.7161442,0.9086222,2.088216,3.076186,1.018289,...,1.918293,0.863867,0.7642734,1.302405,1.180993,1.461404,4.241833,0.06504755,0.4148359,0.01089441
Lead,5.988691,6.450016,5.888161,25.48489,4.51313,6.489717,7.088447,17.51798,24.92614,10.3491,...,12.82917,5.128183,6.759007,10.27748,9.514317,10.68186,35.72923,0.4523222,2.465387,0.125917
WATER LEAK,40.88403,-19.9396,-30.71275,401.7378,-0.8030686,40.11402,68.02848,156.3553,264.3896,6.398351,...,233.1272,110.7555,51.01749,96.9432,104.8749,140.4407,349.0737,8.13101,58.99872,0.06559878
Bus Stop Shelter Placement,0.6706793,0.6926212,0.6291183,2.614048,0.4881913,0.7438327,0.7803264,1.68159,2.602021,1.106566,...,1.481257,0.6196401,0.6941801,1.163748,1.067234,1.144325,3.485508,0.04850229,0.278299,0.01279996
Found Property,-0.001320286,-0.568953,-0.3357697,2.963683,-0.2125555,-0.2902751,-0.04339379,11.26483,1.557988,-0.4055422,...,0.366893,0.4203116,0.6114914,-0.470399,0.1529041,0.7654729,6.313528,0.09807515,0.5421264,0.09202907


In [36]:
predicted_table.to_csv('outputs/predicted_values.csv')
observed_table.to_csv('outputs/observed_values.csv')